In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

In [2]:
title = "tourism"
alpha = 0.3

# Extract data from json file

In [3]:
import matplotlib.pyplot as plt
import glob
from pathlib import Path
from jsonmerge import merge

entries = os.listdir('C:/Users/xyz/Desktop/Tourism/json_files/done2/')

In [4]:
#north india
json_files=[]
for i in entries:
    x = Path('C:/Users/xyz/Desktop/Tourism/json_files/done2/{}/'.format(i))
    inside_data = os.listdir(x)
    y = inside_data[-1]
    z = Path(x).stem
    files = glob.glob("C:/Users/xyz/Desktop/Tourism/json_files/done2/{}/{}/places.json".format(z,y))
    json_files.append(files[0])
    
format_ad=[]
name=[]
for f in json_files:
        data = pd.read_json(f)
        for i in data["formatted_address"]:
            format_ad.append(i)
        for j in data["name"]:
            name.append(j)


In [5]:
#union territory
format_ad_2=[]
name_2=[]

entries = os.listdir('C:/Users/xyz/Desktop/Tourism/json_files/done3/')
json_files=[]
for i in entries:
    x = Path('C:/Users/xyz/Desktop/Tourism/json_files/done3/{}/'.format(i))
    inside_data = os.listdir(x)
    y = inside_data[-1]
    z = Path(x).stem
    files = glob.glob("C:/Users/xyz/Desktop/Tourism/json_files/done3/{}/{}/places.json".format(z,y))
    json_files.append(files[0])
    
for f in json_files:
        data = pd.read_json(f)
        for i in data["formatted_address"]:
            format_ad_2.append(i)
        for j in data["name"]:
            name_2.append(j)

In [6]:
#south india
format_ad_3=[]
name_3=[]

data = pd.read_json("places.json")
for i in data["formatted_address"]:
    format_ad_3.append(i)
for j in data["name"]:
    name_3.append(j)

In [7]:
a = []
b =[]

In [8]:
for i in format_ad:#format_ad,_2,_3
    a.append(i)
for j in name:
    b.append(j)

In [9]:
for i in format_ad_2:#format_ad,_2,_3
    a.append(i)
for j in name_2:
    b.append(j)

In [10]:
for i in format_ad_3:#format_ad,_2,_3
    a.append(i)
for j in name_3:
    b.append(j)

In [11]:
len(b)

5582

# Concat All tourist place name and address into dataset

In [12]:
dataset = []
for j in range(len(a)):
    dataset.append((str(b[j])+" "+ a[j]))

In [13]:
dataset[0]

'Tawang War Memorial Cona, Shannan, Arunachal Pradesh 790104'

In [14]:
N = len(dataset)

In [15]:
# Similarly do and insert into text
text=[]
for i in range(len(a)):
    text.append(b[i]+" " +a[i])

In [16]:
text[0]

'Tawang War Memorial Cona, Shannan, Arunachal Pradesh 790104'

# Preprocessing

In [17]:
#remove functuation
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct


In [18]:
# Removing stopwords
stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(tokenized_list):
    text= [word for word in tokenized_list if word not in stopword]
    return text

In [19]:
# stemming in our sms data
ps = nltk.PorterStemmer()
def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text


In [20]:
#lemmatizing
wn = nltk.WordNetLemmatizer()
def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

In [21]:
#remove_numbers
def remove_num(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list] 
    return list

In [22]:
#Tokenization
import re

def tokenize(text):
    tokens = re.split('\W+',text.lower())
    return tokens


In [23]:
# remove empty set
def remove_empty(x):
    for i in x:
        if len(i)<=2:
            x.remove(i)
    return x

In [24]:
def preprocessing_data(x):
    x = remove_punct(x)
    x = tokenize(x)
    x = remove_num(x)
    x = remove_stopwords(x)
    x =stemming(x)
    x = lemmatizing(x)
    x = remove_empty(x)
    return x

In [25]:
#main point
processed_title = []
processed_text = []
x = -1
for i in dataset[:N]:
    x=x+1
    processed_text.append(preprocessing_data(text[x]))
    processed_title.append(preprocessing_data(i))

In [26]:
processed_title[0]

['tawang', 'war', 'memori', 'cona', 'shannan', 'arunach', 'pradesh']

In [27]:
processed_text[0]

['tawang', 'war', 'memori', 'cona', 'shannan', 'arunach', 'pradesh']

# Calculating df for all words

In [28]:
DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [29]:
total_vocab_size = len(DF)

In [30]:
total_vocab_size

10383

In [31]:
total_vocab = [x for x in DF]

In [32]:
print(total_vocab[:20])

['tawang', 'war', 'memori', 'cona', 'shannan', 'arunach', 'pradesh', 'monasteri', 'bomdila', 'counti', 'jung', 'fall', 'nuranang', 'namdapha', 'nation', 'park', 'director', 'tiger', 'reserv', 'road']


In [33]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

# Calculating TF-IDF for tags , consider this as actual TF-IDF as we will add the title weight to this

In [34]:
doc = 0

tf_idf = {}

for i in range(0,N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1

In [35]:
tf_idf

{(0, 'arunach'): 0.6855485926887522,
 (0, 'cona'): 0.8118632237348457,
 (0, 'memori'): 0.7128421979405678,
 (0, 'pradesh'): 0.2257091891361521,
 (0, 'shannan'): 0.91860813828202,
 (0, 'tawang'): 0.9765317251546151,
 (0, 'war'): 0.8554891736707289,
 (1, 'arunach'): 0.7998066914702109,
 (1, 'cona'): 0.9471737610239865,
 (1, 'monasteri'): 0.8886074465510052,
 (1, 'pradesh'): 0.26332738732551075,
 (1, 'tawang'): 2.2785740253607685,
 (2, 'arunach'): 0.6855485926887522,
 (2, 'bomdila'): 2.06891062405442,
 (2, 'cona'): 0.8118632237348457,
 (2, 'counti'): 0.8554891736707289,
 (2, 'monasteri'): 0.7616635256151474,
 (2, 'pradesh'): 0.2257091891361521,
 (3, 'arunach'): 0.5998550186026582,
 (3, 'cona'): 0.71038032076799,
 (3, 'fall'): 0.9144170614335899,
 (3, 'jung'): 0.9917917955938019,
 (3, 'nuranang'): 0.9917917955938019,
 (3, 'pradesh'): 0.1974955404941331,
 (3, 'shannan'): 0.8037821209967676,
 (4, 'arunach'): 0.4362581953473878,
 (4, 'director'): 0.6380039666251146,
 (4, 'india'): 0.003468523

# Calculate TF-IDF for title our(name of places)

In [36]:
doc = 0

tf_idf_title = {}

for i in range(0,N):
    
    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1)) #numerator is added 1 to avoid negative values
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1

In [37]:
tf_idf[(0,"war")]

0.8554891736707289

In [38]:
tf_idf_title[(0,"war")]

0.8554891736707289

# Merging TF-IDF according to weight

In [39]:
for i in tf_idf:
    tf_idf[i] *= alpha

In [40]:
for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]

In [41]:
len(tf_idf)

46865

# Tf-IDF Matching score

In [194]:
def matching_score(k, query):
    preprocessed_query = preprocessing_data(query)
    tokens = preprocessed_query
    print(tokens)
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    l = []
    g = []
    for i in query_weights[:k]:
        l.append(i[0])
        g.append(i[0])
    print(l)
    return g
    



In [202]:
word = "waterfalls in kerala"
x = word.split(' ') 
result =[]
for i in x:
    result.append(matching_score(300,i))

['waterfal']

[5396, 3994, 3998, 4488, 4747, 2904, 3046, 3997, 4491, 4712, 4869, 4997, 4999, 1047, 2466, 335, 345, 1268, 1694, 1913, 2014, 2027, 2056, 2057, 2069, 2080, 2223, 2611, 2613, 3203, 3995, 4490, 4498, 4499, 4503, 4735, 4752, 4806, 4883, 4986, 5006, 5012, 5043, 5047, 5081, 5196, 5233, 325, 860, 946, 1227, 1768, 1817, 2028, 2306, 2907, 3137, 3140, 3222, 3311, 3492, 3991, 3992, 4312, 4485, 4486, 4487, 4529, 4574, 4672, 5042, 1462, 1562, 1564, 1683, 2026, 4492, 4493, 4497, 4501, 4504, 4708, 4748, 4990, 5045, 5053, 5163, 858, 953, 1118, 1277, 1401, 1671, 1855, 2817, 3102, 3169, 3174, 3243, 4240, 4243, 4502, 4799, 4884, 5044, 5048, 5049, 856, 1658, 1680, 1752, 2621, 4494, 5111, 5123, 405, 1046, 4496, 4500, 4681, 4894, 5119, 5120, 95, 3070, 4751]
[]

[]
['kerala']

[4673, 4276, 4304, 4305, 4306, 4311, 4323, 4429, 4466, 4468, 4478, 4527, 4531, 4562, 4618, 4620, 4631, 4637, 4712, 4739, 4759, 4760, 3149, 4170, 4172, 4308, 4310, 4313, 4345, 4375, 4379, 4395, 4402, 4440, 4457, 4490, 4498

In [203]:
result_set = []
for i in result:
    for j in i:
        result_set.append(j)

In [204]:
def Repeat(x): 
    _size = len(x) 
    repeated = [] 
    for i in range(_size): 
        k = i + 1
        for j in range(k, _size): 
            if x[i] == x[j] and x[i] not in repeated: 
                repeated.append(x[i]) 
    return repeated 

In [205]:
if len(x)>1:
    final = Repeat(result_set)
    for i in final:
        print(dataset[i])

else:
    for i in result_set:
        print(dataset[i])

Thusharagiri Waterfalls Thusharagiri Waterfalls, Thusaragiri, Kerala
Attukad Waterfalls Attukad Waterfall Rd, Pallivasal, Kerala 685565, India
Palaruvi Waterfalls Road to Palaruvi Waterfalls, Kollam, Kerala, India
Palaruvi Waterfalls Road to Palaruvi Waterfalls, Kollam, Kerala, India
Viripara Waterfalls Kerala 685561, India
Soochipara Waterfalls Vellarimala, Kerala 673577, India
Chinnakanel WaterFalls NH-49, Chinnakanal, Kerala 685618, India
Mankayam Waterfalls Brimore, Kerala 695563, India
Adyanpara Waterfall Malappuram, Kerala 679334, India
Pappadi Waterfall Akathethara, Kerala 678008, India
Kuthumkal Waterfalls Kuthumkal, Kerala 685566, India
Marottichal Waterfalls Thrissur, Kerala 680014, India
Dhoni Waterfalls Puthuppariyaram, Palakkad, Kerala 678591, India
Thommankuthu Waterfalls Thodupuzha, Idukki, Kerala 685581, India
Kanthanpara Waterfalls, Meppadi Meppadi, Kerala 673577, India
Valara Waterfalls NH 49, Chillithodu, Kerala 685561, India
Chethalayam Waterfalls Chethalayam, Kidan